In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/X_test_file.pickle
/kaggle/input/y_train_file.pickle
/kaggle/input/y_test_file.pickle
/kaggle/input/X_train_file.pickle


In [2]:
import pickle
import os
print(os.listdir("../input"))


['X_test_file.pickle', 'y_train_file.pickle', 'y_test_file.pickle', 'X_train_file.pickle']


In [3]:
y_train=pd.read_pickle('../input/y_train_file.pickle')
X_train=pd.read_pickle('../input/X_train_file.pickle')
y_test=pd.read_pickle('../input/y_test_file.pickle')
X_test=pd.read_pickle('../input/X_test_file.pickle')

In [4]:
print (y_train.shape)
print (X_train.shape)

(3600000,)
(3600000, 128)


In [5]:
from keras.models import Model, Sequential, Input
from keras.layers import Dense, Embedding, Input, Conv1D, GlobalMaxPool1D, GlobalAveragePooling1D, Dropout, concatenate, Layer, InputSpec, CuDNNLSTM, SpatialDropout1D, Activation, LSTM
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras import activations, initializers, regularizers, constraints, optimizers, layers
from keras.utils.conv_utils import conv_output_length
from keras.regularizers import l2
from keras.constraints import maxnorm
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, SpatialDropout1D, Activation
from keras.layers import Conv1D, Bidirectional, GlobalMaxPool1D, MaxPooling1D, BatchNormalization
from keras.optimizers import Adam


Using TensorFlow backend.


In [6]:
# number of unique words we want to use
max_features = 8192

# max number of words in a comment to use (or: number of columns in incoming embedding vector)
max_len = 128

# dimension of the embedding variable (or: number of rows in output of embedding vector)
embedding_dims = 64

In [7]:
# instantiate CNN model
cnn_2cnv = Sequential()

# add embedding layer 
cnn_2cnv.add(Embedding(input_dim=max_features, input_length=max_len,
                        output_dim=embedding_dims))
 
# set the dropout layer to drop out 50% of the nodes
cnn_2cnv.add(SpatialDropout1D(0.5))

# add convolutional layer that has ...
# ... 100 filters with a kernel size of 4 so that each convolution will consider a window of 4 word embeddings
cnn_2cnv.add(Conv1D(filters=100, kernel_size=4, padding='same', activation='relu'))

# add normalization layer
cnn_2cnv.add(BatchNormalization())

# add convolutional layer that has ...
# ... 100 filters with a kernel size of 4 so that each convolution will consider a window of 4 word embeddings
cnn_2cnv.add(Conv1D(filters=100, kernel_size=4, padding='same', activation='relu'))

# add normalization layer
cnn_2cnv.add(BatchNormalization())

# add pooling layer 
cnn_2cnv.add(GlobalMaxPool1D())

# set the dropout layer to drop out 50% of the nodes
cnn_2cnv.add(Dropout(0.5))

# add dense layer to produce an output dimension of 50 and using relu activation
cnn_2cnv.add(Dense(50, activation='relu'))
# finally add a dense layer
cnn_2cnv.add(Dense(1, activation='sigmoid'))

In [8]:
cnn_2cnv.compile(loss='binary_crossentropy',
                  optimizer=Adam(0.01),
                  metrics=['accuracy'])

cnn_2cnv.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 128, 64)           524288    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 128, 64)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 128, 100)          25700     
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 100)          400       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 128, 100)          40100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 128, 100)          400       
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 100)              

In [ ]:
cnn_2cnv_hist = cnn_2cnv.fit(X_train, y_train, batch_size=512, 
                         epochs=8, validation_split=0.1)

/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 3240000 samples, validate on 360000 samples
Epoch 1/8
3240000/3240000 [==============================] - 3219s 994us/step - loss: 0.2760 - accuracy: 0.8817 - val_loss: 0.2206 - val_accuracy: 0.9118
Epoch 2/8
3240000/3240000 [==============================] - 3235s 998us/step - loss: 0.2331 - accuracy: 0.9061 - val_loss: 0.2106 - val_accuracy: 0.9157
Epoch 3/8
3240000/3240000 [==============================] - 3257s 1ms/step - loss: 0.2251 - accuracy: 0.9099 - val_loss: 0.2124 - val_accuracy: 0.9157
Epoch 4/8
3240000/3240000 [==============================] - 3184s 983us/step - loss: 0.2199 - accuracy: 0.9121 - val_loss: 0.2029 - val_accuracy: 0.9193
Epoch 5/8
3240000/3240000 [==============================] - 3294s 1ms/step - loss: 0.2164 - accuracy: 0.9137 - val_loss: 0.2015 - val_accuracy: 0.9201
Epoch 6/8
3240000/3240000 [==============================] - 3277s 1ms/step - loss: 0.2138 - accuracy: 0.9149 - val_loss: 0.2012 - val_accuracy: 0.9208
Epoch 7/8
2546176/3240000 [==

In [ ]:
filename = 'finalized_model.sav'
pickle.dump(cnn_2cnv, open(filename, 'wb'))

In [ ]:
cnn_2cnv_loss, cnn_2cnv_auc = cnn_2cnv.evaluate(X_test, y_test, batch_size=32)
print('Test Loss:    ', cnn_2cnv_loss)
print('Test Accuracy:', cnn_2cnv_auc)

In [ ]:
cnn_2cnv.save('my_model.h5')

In [ ]:
import joblib
joblib_file = "joblib_RL_Model.pkl"  
joblib.dump(cnn_2cnv, joblib_file)

In [ ]:
import joblib
cnn_model = joblib.load(joblib_file)

In [ ]:
cnn_model.summary()

In [ ]:
cnn_2cnv.evaluate(X_test, y_test, batch_size=32)
